In [1]:
# ! pip install pandas web3 hexbytes rlp fastlz clickhouse-connect
# ! pip install python-dotenv

### Readme
FastLZ needs Python version 3.9x or lower, make sure your environment is using a later python version

In [2]:
import pandas as pd
from web3 import Web3
from hexbytes import HexBytes
import ast
import rlp
from rlp.sedes import Binary, big_endian_int, binary, List
from eth_utils import to_bytes, to_hex
import fastlz
import sys
import os
import dotenv
import time
dotenv.load_dotenv()
sys.path.append("../../helper_functions")
import clickhouse_utils as ch
sys.path.pop()

client = ch.connect_to_clickhouse_db() #Default is OPLabs DB

In [3]:
# Run configs
schemas_to_select = [
        # 'op', 
        # 'base',
        'mode',
        # 'fraxtal',
        'zora'
        ]  # Add more schemas as needed
days_of_data = 28

#FastLZ Regression Metrics
# Specs - https://specs.optimism.io/fjord/exec-engine.html?search=#fjord-l1-cost-fee-changes-fastlz-estimator
intercept = -42_585_600
fastlzCoef = 836_500
minTransactionSize = 100
scaled_by = 1e6

### Execute

In [4]:
# Read the CSV file
csv_path = '../../op_chains_tracking/outputs/chain_metadata.csv'
df = pd.read_csv(csv_path)

# Filter the DataFrame based on the schemas_to_select list
filtered_df = df[df['oplabs_db_schema'].isin(schemas_to_select)]

# Select the required columns and convert to a list of dictionaries
chain_mappings_list = filtered_df[['oplabs_db_schema', 'display_name', 'mainnet_chain_id']].rename(
    columns={'oplabs_db_schema': 'schema_name', 'mainnet_chain_id': 'chain_id'}
).to_dict(orient='records')

# Print the resulting list of dictionaries
# print(chain_mappings_list)

In [5]:
# # Test transaction receipt
# from web3 import Web3
# op_rpc = os.getenv("OP_PUBLIC_RPC")
# w3 = Web3(Web3.HTTPProvider(op_rpc))

# tx_test = '0xcea81f2e836a37b38ba82afd37e6f66c02e348e7b89538aa232013d91edcb926'
# tx = w3.eth.get_transaction(tx_test)
# txr = w3.eth.get_transaction_receipt(tx_test)
# # # txraw = w3.eth.get_raw_transaction(tx_test)
# print(tx)
# # print(txr)
# # # print(txraw)

In [6]:
# may not sufficent due to missing transaction signature fields

# Get L2 Txs from Clickhouse / Goldsky
query_by_day = '''
        SELECT @chain_id@ as chain_id, nonce, gas, max_fee_per_gas, max_priority_fee_per_gas,
                to_address as to, value, input, block_timestamp, block_number, hash, receipt_gas_used
        FROM @chain_db_name@_transactions
        WHERE gas_price > 0
        # 1 day chunk
        AND block_timestamp < DATE_TRUNC('day',NOW()) - interval '@day_num@ days'
        AND block_timestamp >= DATE_TRUNC('day',NOW()) - (interval '@day_num@ days') - (interval '1 day')

        SETTINGS max_execution_time = 3000
'''
# AND hash = '0xcea81f2e836a37b38ba82afd37e6f66c02e348e7b89538aa232013d91edcb926'
# AND block_number = 120731426

# txs_df

In [7]:
# Process transactions and RLP encode
#https://ethereum.org/en/developers/docs/transactions/

# NOTE THE RLP ENCODING IS NOT 1:1 WITH ETHERSCAN YET (but it's ~close-ish)
def process_and_encode_transaction(row):
    try:
        # Check if "to" field is None or empty
        to_field = row["to"]
        if to_field is None or to_field == '':
            to_hexstr = b''
        else:
            try:
                if isinstance(to_field, str):
                    to_hexstr = to_bytes(hexstr=to_field)
                elif isinstance(to_field, bytes):
                    to_hexstr = to_field
                else:
                    raise ValueError(f"Unexpected type for 'to' field: {type(to_field)}")
            except UnicodeDecodeError as e:
                print(f"Error decoding 'to' field: {e}")
                print(f"Problematic byte sequence: {to_field[e.start:e.end]}")
                to_hexstr = b''  # Fallback to empty if there's an error

        tx_params = {
            'chainId': int(row['chain_id']),
            'nonce': int(row['nonce']),
            'maxPriorityFeePerGas': int(row['max_priority_fee_per_gas']),
            'maxFeePerGas': int(row['max_fee_per_gas']),
            'gas': int(row['gas']),
            'to': to_hexstr,
            'value': to_bytes(int(row['value'])),
            'input': HexBytes(row['input']),
            'accessList': row['access_list'],
            'v': int(row['v']),
            'r': HexBytes(row['r']),
            's': HexBytes(row['s'])
        }

        transaction = [
            tx_params['nonce'],
            tx_params['to'],
            tx_params['value'],
            tx_params['gas'],
            tx_params['maxFeePerGas'],
            tx_params['input'],
            tx_params['chainId'],
            tx_params['v'],
            tx_params['r'],
            tx_params['s']
        ]

        encoded_tx = rlp.encode(transaction)
        return Web3.to_hex(encoded_tx), len(encoded_tx)

    except (ValueError, TypeError, UnicodeDecodeError) as e:
        print("Error:", e)
        print("Failed Transaction Info:")
        print(row)
        if 'to_hexstr' in locals():
            print("to_hexstr:", to_hexstr)
        return None, None

# Function to compress transaction data
def compress_transaction(encoded_transaction):

    hex_string = encoded_transaction[2:]
    # Convert the hexadecimal string to bytes
    byte_string = bytes.fromhex(hex_string)
    compressed_data = fastlz.compress(byte_string)

    return compressed_data.hex(), len(compressed_data)
# Define a function to apply to each row of the DataFrame
def process_and_compress_transaction(row):
    encoded_tx = row['encoded_transaction']
    compressed_tx, len_tx = compress_transaction(encoded_tx)
    return compressed_tx, len_tx

In [8]:
dfs = []
for chain in chain_mappings_list:
        for day_num in range(0,days_of_data):
                print(chain['schema_name'] + ' : day ' + str(day_num))
                query_map = query_by_day

                query_map = query_map.replace("@chain_db_name@", chain['schema_name'])
                query_map = query_map.replace("@chain_id@", str(chain['chain_id']))
                query_map = query_map.replace("@day_num@", str(day_num))
                
                query_start_time = time.time()
                result_df = client.query_df(query_map)
                query_end_time = time.time()  # Record the start time
                query_elapsed_time = query_end_time - query_start_time
                print (f"        Query Done: Completed in {query_elapsed_time:.2f} seconds")
                # try:
                # Add Dummy Signature and fields
                result_df['access_list'] = '[]'
                result_df['access_list'] = result_df['access_list'].apply(ast.literal_eval)
                result_df['r'] = '0x6727a53c0972c55923242cea052dc4e1105d7b65c91c442e2741440965eac357'
                result_df['s'] = '0x0a8e71aea623adb7b5562fb9a779634f3b84dad7be1e1f22caaa640db352a6ff'
                result_df['v'] = '55'

                # Assuming `txs_df` is your DataFrame
                result_df[['encoded_transaction', 'len_encoded_transaction']] = result_df.apply(process_and_encode_transaction, axis=1, result_type='expand')
                enc_end_time = time.time()  # Record the start time
                enc_elapsed_time = enc_end_time - query_end_time
                print (f"        Encoding Done: Completed in {enc_elapsed_time:.2f} seconds")

                # Apply compression to each transaction in the DataFrame
                result_df[['compressed_transaction', 'compressed_transaction_length']] = result_df.apply(process_and_compress_transaction, axis=1, result_type='expand')
                comp_end_time = time.time()
                comp_elapsed_time = comp_end_time - enc_end_time
                print (f"        Compression Done: Completed in {enc_elapsed_time:.2f} seconds")
                
                # Calculate estimated size for each row
                result_df['estimatedSize_raw'] = result_df.apply(lambda row: (intercept + (row['compressed_transaction_length'] * fastlzCoef)) / scaled_by, axis=1)
                # Calculate minimum value for 'estimatedSize' column
                result_df['estimatedSize'] = result_df.apply(lambda row: max(minTransactionSize, row['estimatedSize_raw']), axis=1)
                est_end_time = time.time()
                est_elapsed_time = est_end_time - comp_end_time
                print (f"        Estimation Done: Completed in {est_elapsed_time:.2f} seconds")

                # Agg L2
                # Convert block_timestamp to date (truncate to day)
                result_df['block_date'] = pd.to_datetime(result_df['block_timestamp']).dt.date
                grouped_df = result_df.groupby(['block_date', 'chain_id'])
                # Define aggregation functions
                agg_functions = {
                        'len_encoded_transaction': ['sum', 'mean', 'count'],
                        'compressed_transaction_length': ['sum', 'mean'],
                        'estimatedSize': ['sum', 'mean']
                }
                # Perform aggregation
                aggregated_df = grouped_df.agg(agg_functions).reset_index()
                # Rename columns for clarity
                aggregated_df.columns = ['block_date', 'chain_id', 
                                        'total_len_encoded_transaction', 'average_len_encoded_transaction', 'transaction_count',
                                        'total_len_compressed_transaction','average_len_compressed_transaction',
                                        'total_estimatedSize', 'average_estimatedSize']
                try:
                        aggregated_df['chain_name'] = chain['schema_name']
                        dfs.append(aggregated_df)
                except:
                        print('nothing to append')
                        continue

aggregated_df = pd.concat(dfs)

zora : day 0
        Query Done: Completed in 1.70 seconds
        Encoding Done: Completed in 7.33 seconds


/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_73273/3350871285.py:69: DeprecationWarning: PY_SSIZE_T_CLEAN will be required for '#' formats
  compressed_data = fastlz.compress(byte_string)


        Compression Done: Completed in 7.33 seconds
        Estimation Done: Completed in 0.87 seconds
zora : day 1
        Query Done: Completed in 9.97 seconds
        Encoding Done: Completed in 6.60 seconds


/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_73273/3350871285.py:69: DeprecationWarning: PY_SSIZE_T_CLEAN will be required for '#' formats
  compressed_data = fastlz.compress(byte_string)


        Compression Done: Completed in 6.60 seconds
        Estimation Done: Completed in 0.81 seconds
zora : day 2
        Query Done: Completed in 4.50 seconds
        Encoding Done: Completed in 6.85 seconds


...

        Compression Done: Completed in 13.98 seconds
        Estimation Done: Completed in 1.79 seconds
mode : day 26
        Query Done: Completed in 6.28 seconds
        Encoding Done: Completed in 11.80 seconds


/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_73273/3350871285.py:69: DeprecationWarning: PY_SSIZE_T_CLEAN will be required for '#' formats
  compressed_data = fastlz.compress(byte_string)


        Compression Done: Completed in 11.80 seconds
        Estimation Done: Completed in 1.51 seconds
mode : day 27
        Query Done: Completed in 7.41 seconds
        Encoding Done: Completed in 19.72 seconds


/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_73273/3350871285.py:69: DeprecationWarning: PY_SSIZE_T_CLEAN will be required for '#' formats
  compressed_data = fastlz.compress(byte_string)


        Compression Done: Completed in 19.72 seconds
        Estimation Done: Completed in 2.40 seconds


In [9]:
# print(aggregated_df['encoded_transaction'][0])
# print(len(aggregated_df['encoded_transaction'][0]))

In [10]:
# Calculate weighted averages and mean
def weighted_avg(df, value_column, weight_column):
    return (df[value_column] * df[weight_column]).sum() / df[weight_column].sum()


In [15]:
# aggregated_df

In [16]:
agg_cols = ['average_len_encoded_transaction','average_estimatedSize','transaction_count']
grouped_df = aggregated_df.groupby(['chain_id','chain_name'])
total_aggregated_df = grouped_df.apply(
    lambda x: pd.Series({
        'average_len_encoded_transaction': weighted_avg(x, 'average_len_encoded_transaction', 'transaction_count'),
        'average_estimatedSize': weighted_avg(x, 'average_estimatedSize', 'transaction_count'),
        'transaction_count': x['transaction_count'].mean()
    })
).reset_index()
total_aggregated_df
total_aggregated_df =total_aggregated_df.reset_index()
total_aggregated_df

/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_73273/2802653718.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  total_aggregated_df = grouped_df.apply(


,index,chain_id,chain_name,average_len_encoded_transaction,average_estimatedSize,transaction_count
0,0,34443.0,mode,247.605537,131.539685,209725.714286
1,1,7777777.0,zora,483.248153,171.836547,91868.392857


In [17]:
from datetime import datetime
# Generate current timestamp
current_timestamp = datetime.now().strftime("%Y%m%d_%H%M")
# Define the file path
file_path = f"outputs/l2_output_{current_timestamp}.csv"
total_file_path = f"outputs/total_l2_output_{current_timestamp}.csv"
# Save the DataFrame to CSV
aggregated_df.to_csv(file_path, index=False)
total_aggregated_df.to_csv(total_file_path, index=False)
print(f"DataFrame saved to: {file_path}")

DataFrame saved to: outputs/l2_output_20240606_2030.csv


In [ ]:
# Pull aggregate L1 data

In [ ]:
# Generate L2 : L1 ratio metrics